In [13]:
import os
import cv2 as cv
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from math import log10, sqrt 
import tensorflow as tf


def load_images_from_folder(foldername, target_size=(64, 64)):
    images = []
    for filename in os.listdir(foldername):
        img = cv.imread(os.path.join(foldername, filename), cv.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv.resize(img, target_size)  
            images.append(img)
    return images
# https://stackoverflow.com/questions/66471454/how-to-read-images-from-our-system-folder-to-jupyter-notebook-python-file

# Load images
images=load_images_from_folder("pneumonia")
images = np.array(images)
images = images.astype('float32') / 255.
images = np.expand_dims(images, axis=-1)

lam25noise = np.random.poisson(25, images.shape)
lam25 = images + lam25noise

lam50noise = np.random.poisson(50, images.shape)
lam50 = images + lam50noise

lam75noise = np.random.poisson(75, images.shape)
lam75 = images + lam75noise
# https://stackoverflow.com/questions/19289470/adding-poisson-noise-to-an-image

# Split the dataset into training and testing sets (80% train, 20% test)
x_train, x_test = train_test_split(images, test_size=0.2, random_state=42)
train25, test25 = train_test_split(lam25, test_size=0.2, random_state=42)
train50, test50 = train_test_split(lam50, test_size=0.2, random_state=42)
train75, test75 = train_test_split(lam75, test_size=0.2, random_state=42)

def PSNR(original, compressed):
    original = tf.cast(original, tf.float32)
    compressed = tf.cast(compressed, tf.float32)      
    mse = tf.reduce_mean(tf.square(original - compressed))
    max_pixel = 255.0
    psnr = 20 * tf.math.log(max_pixel / tf.sqrt(mse)) / tf.math.log(10.0)

    return psnr 
# PSNR function from previous submission, modified for Tensor use 

input_img = Input(shape=(64, 64, 1))  

def forward(input):
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
   
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    return decoded
     # https://www.tensorflow.org/api_docs/python/tf

# Lambda = 25

In [14]:
model25 = Model(input_img, forward(input_img))
model25.compile(optimizer='adam', loss=PSNR)

model25.fit(train25, x_train,
                epochs=5,
                batch_size=128,
                shuffle=True,
                validation_data=(test25, x_test))

psnr25 = model25.evaluate(test25, x_test)
print("PSNR:", psnr25)
#https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/

Epoch 1/5
15/15 [==============================] - 5s 344ms/step - loss: 53.5378 - val_loss: 53.5676
Epoch 2/5
15/15 [==============================] - 5s 331ms/step - loss: 53.5208 - val_loss: 53.5676
Epoch 3/5
15/15 [==============================] - 5s 315ms/step - loss: 53.5214 - val_loss: 53.5676
Epoch 4/5
15/15 [==============================] - 4s 298ms/step - loss: 53.5208 - val_loss: 53.5676
Epoch 5/5
15/15 [==============================] - 0s 20ms/step - loss: 53.5707
PSNR: 53.57073974609375


# Lambda = 50

In [15]:
model50 = Model(input_img, forward(input_img))
model50.compile(optimizer='adam', loss=PSNR)

model50.fit(train50, x_train,
                epochs=5,
                batch_size=128,
                shuffle=True,
                validation_data=(test50, x_test))

psnr50 = model50.evaluate(test50, x_test)
print("PSNR:", psnr50)

Epoch 1/5
15/15 [==============================] - 5s 299ms/step - loss: 53.0503 - val_loss: 52.9975
Epoch 2/5
15/15 [==============================] - 5s 311ms/step - loss: 53.0395 - val_loss: 52.9975
Epoch 3/5
15/15 [==============================] - 4s 298ms/step - loss: 53.0395 - val_loss: 52.9975
Epoch 4/5
15/15 [==============================] - 5s 305ms/step - loss: 53.0391 - val_loss: 52.9975
Epoch 5/5
15/15 [==============================] - 0s 27ms/step - loss: 52.9998
PSNR: 52.999839782714844


# Lambda = 75

In [16]:
model75 = Model(input_img, forward(input_img))
model75.compile(optimizer='adam', loss=PSNR)

model75.fit(train75, x_train,
                epochs=5,
                batch_size=128,
                shuffle=True,
                validation_data=(test75, x_test))

psnr75 = model75.evaluate(test75, x_test)
print("PSNR:", psnr75)

Epoch 1/5
15/15 [==============================] - 5s 332ms/step - loss: 53.0660 - val_loss: 52.9975
Epoch 2/5
15/15 [==============================] - 5s 346ms/step - loss: 53.0396 - val_loss: 52.9975
Epoch 3/5
15/15 [==============================] - 5s 322ms/step - loss: 53.0394 - val_loss: 52.9975
Epoch 4/5
15/15 [==============================] - 5s 333ms/step - loss: 53.0399 - val_loss: 52.9975
Epoch 5/5
15/15 [==============================] - 0s 20ms/step - loss: 52.9998
PSNR: 52.999839782714844


### Mean Blur

In [17]:
x = 0
for i in test25:
        for n in range(1, 31, 2):
                blur25 = cv.blur(test25[x], (n,n))
                blur50 = cv.blur(test50[x], (n,n))
                blur75 = cv.blur(test75[x], (n,n))
        x+=1


psnr_blur25 = PSNR(test25, blur25)
psnr_blur25 = np.mean(psnr_blur25)
print (psnr_blur25)

psnr_blur50 = PSNR(test50, blur50)
psnr_blur50 = np.mean(psnr_blur50)
print (psnr_blur50)

psnr_blur75 = PSNR(test75, blur75)
psnr_blur75 = np.mean(psnr_blur75)
print (psnr_blur75)



34.12184
31.114208
29.364433


### Median Blur

In [18]:
x = 0
for i in test25:
        for n in range(1, 31, 2):
                src25 =test25[x].astype(np.uint8)
                src50 = test50[x].astype(np.uint8)
                src75 = test75[x].astype(np.uint8)
                medBlur25 = cv.medianBlur(src25, n)
                medBlur50 = cv.medianBlur(src50, n)
                medBlur75 = cv.medianBlur(src75, n)
        x+=1

psnr_medBlur25 = PSNR(test25, medBlur25)
psnr_medBlur25 = np.mean(psnr_medBlur25)
print (psnr_medBlur25)

psnr_medBlur50 = PSNR(test50, medBlur50)
psnr_medBlur50 = np.mean(psnr_medBlur50)
print (psnr_medBlur50)

psnr_medBlur75 = PSNR(test75, medBlur75)
psnr_medBlur75 = np.mean(psnr_medBlur75)
print (psnr_medBlur75)

34.032207
31.050962
29.305325


### Bilateral Filter

In [19]:
x = 0
for i in test25:
        for n in range(1, 31, 2):
                src25 =test25[x].astype(np.uint8)
                src50 = test50[x].astype(np.uint8)
                src75 = test75[x].astype(np.uint8)
                bilateral25 = cv.bilateralFilter(src25, n, n * 2, n / 2)
                bilateral50 = cv.bilateralFilter(src50, n, n * 2, n / 2)
                bilateral75 = cv.bilateralFilter(src75, n, n * 2, n / 2)
        x += 1

psnr_bilateral25 = PSNR(test25, bilateral25)
psnr_bilateral25 = np.mean(psnr_bilateral25)
print (psnr_bilateral25)

psnr_bilateral50 = PSNR(test50, bilateral50)
psnr_bilateral50 = np.mean(psnr_bilateral50)
print (psnr_bilateral50)

psnr_bilateral75 = PSNR(test75, bilateral75)
psnr_bilateral75 = np.mean(psnr_bilateral75)
print (psnr_bilateral75)

34.09472
31.075706
29.330585


### Gaussian Blur

In [20]:
x = 0
for i in test25:
    for n in range(1, 31, 2):
        src25 =test25[x].astype(np.uint8)
        src50 = test50[x].astype(np.uint8)
        src75 = test75[x].astype(np.uint8)
        gaussian25 = cv.GaussianBlur(src25, (n, n), 0)
        gaussian50 = cv.GaussianBlur(src50, (n, n), 0)
        gaussian75 = cv.GaussianBlur(src75, (n, n), 0)        
    x += 1

psnr_gaussian25 = PSNR(test25, gaussian25)
psnr_gaussian25 = np.mean(psnr_gaussian25)
print (psnr_gaussian25)

psnr_gaussian50 = PSNR(test50, gaussian50)
psnr_gaussian50 = np.mean(psnr_gaussian50)
print (psnr_gaussian50)

psnr_gaussian75 = PSNR(test75, gaussian75)
psnr_gaussian75 = np.mean(psnr_gaussian75)
print (psnr_gaussian75)

34.073586
31.052164
29.320675


In [21]:
autoencoder_lambdas = [25, 50, 75]

print("\t\t\tLambda =", autoencoder_lambdas[0], "\t\tLambda =", autoencoder_lambdas[1], "\t\tLambda =", autoencoder_lambdas[2])
print("Median Filter\t\t", psnr_medBlur25, "\t\t", psnr_medBlur50, "\t\t", psnr_medBlur75)
print("Mean Filter\t\t", psnr_blur25, "\t\t", psnr_blur50, "\t\t", psnr_blur75)
print("Bilateral Filter\t", psnr_bilateral25, "\t\t", psnr_bilateral50, "\t\t", psnr_bilateral75)
print("Gaussian Filter\t\t", psnr_gaussian25, "\t\t", psnr_gaussian50, "\t\t", psnr_gaussian75)
print("Autoencoder Model\t", psnr25, "\t", psnr50, "\t", psnr75)

			Lambda = 25 		Lambda = 50 		Lambda = 75
Median Filter		 34.032207 		 31.050962 		 29.305325
Mean Filter		 34.12184 		 31.114208 		 29.364433
Bilateral Filter	 34.09472 		 31.075706 		 29.330585
Gaussian Filter		 34.073586 		 31.052164 		 29.320675
Autoencoder Model	 53.57073974609375 	 52.999839782714844 	 52.999839782714844


#### a) Why is poisson distribution the ideal one to use to simulate noise for medical images? Why not gaussian or something else? Answer in terms of relevance of medical applications.

Poisson distribution is ideal since the noise it models is not additive. Poisson noise is signal dependent which means it can capture better the effect of the light in the photograph which is what causes noise. The nature of poisson distribution, which counts events over a period of time also reflects the interaction between the photon beams and certain parts of the body; determining which areas recieved low or high photons.

https://www.proquest.com/openview/e96ddb9f4d6900580e8354a09ff5ff98/1?pq-origsite=gscholar&cbl=1616336

#### b) Which one performed the best? Why do you think this is the case?



Based on the average PSNR loss values provided:

- For Lambda = 25, the autoencoder model performed the best with a PSNR of 53.57
- For Lambda = 50, the autoencoder model performed the best with a PSNR of 52.99
- For Lambda = 75, the autoencoder model performed the best with a PSNR of 52.99.

The autoencoder model performed the best across all Lambda values. Unlike classical methods such as the Median, Mean, Bilateral, and Gaussian filtering––which apply fixed rules to filter noise––autoencoders adaptively learn from the noisy data during training. This allows them to outperform classical methods, especially when dealing with varying noise distributions or characteristics in the data. Consequently, the autoencoder model achieved higher PSNR values and a lower PSNR loss, indicating signs of superior denoising performance compared to the aforementioned classical methods.